# Strands Agents 시작하기

이 워크샵은 도구 통합 기능을 갖춘 AI 에이전트 구축을 위한 강력한 프레임워크인 Strands Agents를 소개합니다. 에이전트 생성, 내장 도구 사용, 커스텀 도구 개발의 기본 사항을 배우게 됩니다.

## 개요

이 실습에서는 다음을 수행합니다:
- Strands Agents의 핵심 개념 이해
- 내장 도구를 사용한 첫 번째 AI 에이전트 생성
- 특정 사용 사례를 위한 커스텀 도구 구축
- 대화 기록 및 에이전트 메모리 탐색
- 에이전트 개발 모범 사례 학습

## 사전 요구사항

이 실습을 시작하기 전에 다음 사항을 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Strands Agents에 필요한 패키지를 설치하세요:

In [ ]:
#%pip install -q strands-agents strands-agents-tools rich

AWS 리전에 따라 Bedrock 이 제공하는 model 과 ID 가 다릅니다. 아래 셀을 실행하여 Nova Pro 모델 ID를 설정하세요:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## Strands Agents란 무엇인가요?

Strands Agents는 도구 통합 기능을 갖춘 AI 에이전트 생성을 단순화하는 Python 프레임워크입니다. 주요 기능은 다음과 같습니다:

- **간단한 에이전트 생성**: 최소한의 코드로 AI 에이전트를 생성할 수 있는 사용하기 쉬운 API
- **내장 도구**: 계산기, 웹 검색 등과 같은 사전 구축된 도구
- **커스텀 도구 지원**: 간단한 Python 함수로 자신만의 도구 생성
- **대화 메모리**: 자동 대화 기록 관리
- **모델 유연성**: Amazon Bedrock 및 OpenAI의 모델을 포함한 다양한 언어 모델 지원

Strands Agents는 외부 시스템과 상호 작용하고 복잡한 작업을 수행할 수 있는 정교한 AI 애플리케이션 구축을 위한 기반을 제공합니다.

## 첫 번째 Strands Agent 생성하기

내장 계산기 도구를 사용하여 간단한 에이전트를 생성하는 것부터 시작해보겠습니다. 이는 Strands Agents의 기본 구조와 도구가 통합되는 방식을 보여줍니다:

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# 첫 번째 에이전트 생성
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="당신은 간결한 답변을 제공하는 도움이 되는 도우미입니다.",
    tools=[calculator],
)

agent("반지름이 8.26 cm인 원의 넓이는 얼마입니까?")

## 커스텀 도구 구축하기

위에서는 Strands SDK 의 내장 도구를 사용했는데 사용자 정의 도구를 생성할 수도 있습니다. 
Strands Agents의 강력한 기능 중 하나는 커스텀 도구를 생성할 수 있는 능력입니다. 날씨 도구를 생성하고 내장 계산기와 결합해보겠습니다:

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# 데모용 커스텀 날씨 도구 생성
@tool
def weather(city: str) -> str:
    """도시의 날씨 정보를 가져옵니다
    Args:
        city: 도시 또는 위치 이름
    """
    return f"{city}의 날씨: 맑음, 35°C" # 데모용 더미 결과

# 첫 번째 에이전트 생성
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="You are a helpful assistant that provides concise responses.",
    tools=[weather, calculator],
)

agent("홍콩 날씨는 어떤가요? 화씨로 온도를 알려주세요.")

## 에이전트 실행 흐름 이해하기

위에서는 두개의 도구 (내장도구, 사용자도구) 를 모델이 스스로 (agentic) 판단하여 어떤 도구를 사용할지 선택하는 것을 보았습니다. 
이번에는 Strands Agents가 에이전트 루프를 통해 요청을 처리하는 방식을 살펴보겠습니다. 
이는 에이전트 프레임워크의 내부 작동 방식을 이해하는 데 도움이 됩니다:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 횟수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content] # message["content"] 안에서 "text" 키가 있는 항목의 값만 모아 리스트로 생성합니다.
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## 에이전트 루프

에이전트 루프는 추론, 도구 사용, 응답 생성의 순환을 통해 지능적이고 자율적인 행동을 가능하게 하는 Strands Agents SDK의 핵심 개념입니다.

![strands-agents-agent-loop](images/strands-agents-agent-loop.png)

에이전트 루프는 기본적으로 다음 단계를 따릅니다:

1. 사용자 입력 및 컨텍스트 정보 수신
2. 언어 모델(LLM)을 사용하여 입력 처리
3. 정보 수집 또는 작업 수행을 위해 도구를 사용할지 결정
4. 도구 실행 및 결과 수신
5. 새로운 정보로 추론 계속
6. 최종 응답 생성 또는 루프를 통해 다시 반복
   
이 순환은 단일 사용자 상호작용 내에서 여러 번 반복될 수 있어, 에이전트가 복잡한 다단계 추론과 자율적인 행동을 수행할 수 있게 합니다.

참고: [Strands Agents - Agent Loop](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/agent-loop/)


## Strands Agents 대화 관리

Strands Agents는 기본 `SlidingWindowConversationManager` 전략을 사용한 내장 대화 관리를 포함합니다. 

이 시스템은 컨텍스트 관리, 메모리 최적화, 대화 흐름을 처리하여 에이전트가 모델 컨텍스트 제한을 준수하면서 일관된 장기 상호작용을 유지할 수 있도록 합니다. 

이는 세션 내에서 대화 컨텍스트를 자동으로 유지합니다.

**참고:** [Strands Agents - Conversation Management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

In [ ]:
agent("우리가 무엇에 대해 이야기했나요?")

Strands Agents가 대화 기록을 관리하는 방식을 살펴보겠습니다. 이는 에이전트 프레임워크의 내부 작동 방식을 이해하는 데 도움이 됩니다:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 횟수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

### 새 세션에서 Strands Agents 대화 관리 살펴보기

Strands Agents 대화 관리는 세션 내에서 입력 프롬프트에 사용자와 에이전트 메시지를 누적하여 대화 기록을 유지합니다. 

대화 기록은 메모리에 저장되며 세션이 종료되면 휘발됩니다. 즉 새로운 agent 를 만들면 이전 대화기록이 없이 생성됩니다.

새 에이전트를 초기화하여 새 세션을 시뮬레이션하고 Strands Agents 대화 관리 동작을 테스트해보겠습니다.

In [ ]:
# 새 세션을 위한 새 에이전트 초기화
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="당신은 간결한 답변을 제공하는 친절한 조력자입니다.",
    tools=[weather, calculator],
)

agent("우리가 무엇에 대해 이야기했나요?")

에이전트가 홍콩 날씨 문의에 대한 과거 대화를 잊어버리는 것을 볼 수 있습니다. 

이는 Strands Agents 내장 대화 관리가 단기 메모리만 지원한다는 것을 보여줍니다. 

세션과 에이전트 간의 장기 메모리를 활성화하려면, 나중에 [Lab 6: Bedrock AgentCore Memory를 사용한 Strands Agents](../06-bedrock-agentcore-memory/06-agentcore-memory.ipynb)에서 관리형 에이전트 메모리 서비스인 Bedrock AgentCore Memory를 탐색하게 됩니다.

## 요약

이 Strands Agents 소개에서 다음을 배웠습니다:

### 달성한 것
- **첫 번째 AI 에이전트 생성** - 내장 도구(계산기) 사용
- **커스텀 도구 구축** - 특정 사용 사례를 위해 `@tool` 데코레이터 사용
- **대화 관리 탐색** - 다양한 ConversationManager 유형 사용
- **에이전트 실행 흐름 검토** - 메시지 기록 추적

### 추가 Strands 기능
우리가 다룬 내용 외에도 Strands Agents는 다음을 제공합니다:

- **모델 컨텍스트 프로토콜(MCP) 지원**: 확장된 도구 기능을 위한 MCP 서버와의 통합
- **멀티 에이전트 패턴**: 복잡한 워크플로우를 위한 여러 에이전트 조정
- **세션 관리**: 지속적인 대화 저장 및 검색
- **스트리밍 응답**: 더 나은 사용자 경험을 위한 실시간 응답 생성
- **커스텀 모델 통합**: Amazon Bedrock 외의 다양한 LLM 제공업체 지원

### 다음 단계: Bedrock AgentCore 통합

이제 Strands Agents의 기본 사항을 이해했으므로, **Amazon Bedrock AgentCore**와 통합하여 이러한 기능을 향상시키는 방법을 탐색하겠습니다. 이 통합은 다음을 제공합니다:

- **코드 인터프리터**: 에이전트 내에서 Python 코드를 동적으로 실행
- **브라우저 자동화**: 웹 상호작용 및 데이터 추출 기능
- **보안 자격 증명 관리**: 외부 API 키 및 비밀의 안전한 처리
- **런타임 배포**: 클라우드 환경에서 확장 가능한 에이전트 배포
- **향상된 관찰 가능성**: 프로덕션 에이전트를 위한 모니터링 및 디버깅 도구